## 事前準備

ローカルのVSCodeで実行する想定。

まず以下のコマンドでnotebook実行用の仮想環境構築。

```bash
$ cd /path/to/oasobi
$ python -m venv .venv.notebooks.vol1
$ source .venv.notebooks.vol1/bin/activate
$ pip install -r notebooks/vol1/requirements.txt
```

仮想環境を構築したらVSCode上で上記カーネルを選択。

## 検証

Cline, Gemini, LangChainのドキュメントを読んで個人的にLLMは

- チャットによる技術相談
- 自然言語データをJSONや表形式などの構造化データに変換
- ときどき分類タスク

に使用するものと勝手に解釈することにした。

そこで気になったのが大量の**指定したラベル**の中から条件に一致するラベルを正しく分類出来るのかという点。

分類の実験に使える良い問題がないかChatGPTに相談したところ、PyPIに登録されてる全パッケージの名前が列挙されているページを教えてもらった。

(以下のページは巨大であるため、開いてからしばらくするとブラウザがクラッシュするので注意)

https://pypi.org/simple/

これを使えば自分がよく知っている領域で分類タスクを評価できるので早速試してみる。

ついでにパッケージ一覧をファイル出力するコードを作ってもらって以下にコピペ＆実行。

ここで作成したファイルはプロンプト時に分類ラベルとして渡す。

In [1]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path

# PyPI SimpleインデックスのURL
url = "https://pypi.org/simple/"

# リクエストを送ってHTMLを取得
response = requests.get(url)
response.raise_for_status()

# BeautifulSoupでHTMLを解析
soup = BeautifulSoup(response.text, 'html.parser')

# すべての<a>タグからパッケージ名を抽出
packages = [a.text for a in soup.find_all('a')]

# 出力先ファイルパス（ホームディレクトリに保存）
output_path = Path.home() / "all_packages.txt"

# ファイルに保存
with open(output_path, "w", encoding="utf-8") as f:
    for package in packages:
        f.write(package + "\n")

print(f"{len(packages)} 個のパッケージ名を all_packages.txt に保存しました。")

628595 個のパッケージ名を all_packages.txt に保存しました。


上記出力結果より、かなり巨大な分類のテストが出来そう。

あとはChatGPTに生成したファイルと共に以下のプロンプトを入力として実行。

```
all_packages.txtには1行ごとにPythonのパッケージ名が記述されている
この中から{ニッチな技術トピック}に関連するPythonパッケージTOP5
```

次にプロンプトを新規に開始して、all_packages.txtを渡さずに以下のプロンプトを入力＆実行。

```
{ニッチな技術技術トピック}に関連するPythonパッケージTOP5
```

## 結果

上記プロンプトでは`{ニッチな技術トピック}`とぼかしているが、このキーワードはある会社が開発した特定のパッケージが必ず上位に来ると予想して選んだものになる。

なお、all_packages.txtを使用した場合の推薦リストは全くのハズレだったのに対して、テキストだけの指示では上位が完璧な正解という結果になった。

試しにプロンプトを変更して

```
{会社名}が開発した{ニッチな技術技術トピック}に関連するPythonパッケージTOP5
```

とさらにヒントを与えて試したが結果は同じだった。

気になる点として、all_packages.txtを渡さないケースでは`ウェブを検索...`みたいな表示がされたので、もしかしたらRAGが影響しているのではとも思った。

個人的にはこちらが指定したラベルの中から正しいものを選んでくれないとシステム的に制御が難しいので残念な結果。

もっとも少ない実験で決めつけるのもどうかと思うが。

あとGeminiなり他のモデルを使えば結果変わるかも。